In [ ]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
import optuna

# Function to compress an image using JPEG 2000
def compress_image_with_jpeg2000(image, output_path, compression_ratio):
    """
    Compresses an image using JPEG 2000 with the given compression ratio.
    """
    image.save(output_path, "JPEG2000", quality_layers=[compression_ratio])

# Function to calculate compression ratio
def calculate_compression_ratio(original_size, compressed_size):
    """
    Calculates the compression ratio.
    """
    return original_size / compressed_size

# Objective function for Bayesian optimization
def objective(trial, original_image, input_path, output_folder):
    """
    Objective function to optimize compression ratio and image quality.
    """
    # Define hyperparameters to tune
    compression_ratio = trial.suggest_float('compression_ratio', 10, 90)  # 10% to 90%

    # Define the output path for the compressed image
    output_path = os.path.join(output_folder, os.path.basename(input_path).replace(".png", ".jp2"))

    # Compress the image using the chosen hyperparameters
    compress_image_with_jpeg2000(original_image, output_path, compression_ratio)

    # Load the compressed image
    compressed_image = Image.open(output_path)

    # Calculate image quality (e.g., PSNR)
    quality = psnr(np.array(original_image), np.array(compressed_image))

    # Get the original and compressed file sizes
    original_size = os.path.getsize(input_path)
    compressed_size = os.path.getsize(output_path)

    # Calculate compression ratio
    compression_ratio_value = calculate_compression_ratio(original_size, compressed_size)

    # Return the negative quality (since Optuna minimizes the objective)
    return -quality, compression_ratio_value

# Function to compress the dataset
def compress_dataset(input_folder, output_folder, n_trials=50):
    """
    Compresses all images in a dataset using Bayesian optimization.
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the dataset
    images = []
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            img = Image.open(input_path)
            images.append((input_path, img))

    # Run Bayesian optimization for each image
    compression_ratios = []
    for input_path, img in images:
        def wrapped_objective(trial):
            return objective(trial, img, input_path, output_folder)

        # Run the optimization
        study = optuna.create_study(directions=["minimize", "maximize"])  # Minimize negative quality, maximize compression ratio
        study.optimize(wrapped_objective, n_trials=n_trials)

        # Get the best hyperparameters
        best_params = study.best_trials[0].params
        best_compression_ratio = study.best_trials[0].values[1]

        # Compress the image with the best hyperparameters
        output_path = os.path.join(output_folder, os.path.basename(input_path).replace(".png", ".jp2"))
        compress_image_with_jpeg2000(img, output_path, best_params['compression_ratio'])

        # Store the compression ratio
        compression_ratios.append(best_compression_ratio)

        print(f"Compressed {os.path.basename(input_path)} with compression ratio: {best_compression_ratio:.2f}")

    # Calculate the average compression ratio
    avg_compression_ratio = np.mean(compression_ratios)
    print(f"\nAverage Compression Ratio: {avg_compression_ratio:.2f}")

# Example usage
input_folder = "dataset"  # Replace with your dataset path
output_folder = "comprtryy"   # Replace with your output folder
n_trials = 50  # Number of trials for Bayesian optimization

compress_dataset(input_folder, output_folder, n_trials)

[I 2025-03-17 23:51:05,920] A new study created in memory with name: no-name-7ddfc7b2-5421-4ecd-99c6-926fac49d673
[I 2025-03-17 23:51:11,093] Trial 0 finished with values: [-40.971359072524464, 6.347454879760003] and parameters: {'compression_ratio': 61.46219144894481}. 
[I 2025-03-17 23:51:16,397] Trial 1 finished with values: [-43.46440623462443, 1.8623808797066934] and parameters: {'compression_ratio': 18.053056411262986}. 
[I 2025-03-17 23:51:21,812] Trial 2 finished with values: [-44.78389534890597, 1.4110758116763502] and parameters: {'compression_ratio': 13.680952862978746}. 
[I 2025-03-17 23:51:26,826] Trial 3 finished with values: [-42.33491667750521, 2.981330160710613] and parameters: {'compression_ratio': 28.90676495996944}. 
[I 2025-03-17 23:51:31,596] Trial 4 finished with values: [-40.44952582249575, 9.087800180809904] and parameters: {'compression_ratio': 88.11981760455608}. 
[I 2025-03-17 23:51:36,364] Trial 5 finished with values: [-40.634224061565874, 7.85653069657968

Compressed A01_1.jpg with compression ratio: 6.35


[I 2025-03-17 23:55:23,231] Trial 0 finished with values: [-41.4010215616938, 6.279811066192945] and parameters: {'compression_ratio': 62.29300227155683}. 
[I 2025-03-17 23:55:28,150] Trial 1 finished with values: [-42.899621773554586, 2.7708966052233817] and parameters: {'compression_ratio': 27.48503451501182}. 
[I 2025-03-17 23:55:32,879] Trial 2 finished with values: [-41.51953168286305, 5.807276712527285] and parameters: {'compression_ratio': 57.596390525726456}. 
[I 2025-03-17 23:55:37,639] Trial 3 finished with values: [-41.62117617184978, 5.427889336629304] and parameters: {'compression_ratio': 53.78041595836834}. 
[I 2025-03-17 23:55:42,369] Trial 4 finished with values: [-41.76628606905175, 4.943867528987672] and parameters: {'compression_ratio': 49.03358531291002}. 
[I 2025-03-17 23:55:47,353] Trial 5 finished with values: [-43.392586260336685, 2.197187394248851] and parameters: {'compression_ratio': 21.79280980007924}. 
[I 2025-03-17 23:55:52,016] Trial 6 finished with value

Compressed A01_2.jpg with compression ratio: 6.28


[I 2025-03-18 00:00:57,379] Trial 0 finished with values: [-44.34413533013221, 4.7516094533101265] and parameters: {'compression_ratio': 53.3413310279071}. 
[I 2025-03-18 00:01:01,918] Trial 1 finished with values: [-46.00179604949936, 2.428440370905443] and parameters: {'compression_ratio': 27.263495902969268}. 
[I 2025-03-18 00:01:06,192] Trial 2 finished with values: [-44.266197888116274, 4.960300520788867] and parameters: {'compression_ratio': 55.65889378709912}. 
[I 2025-03-18 00:01:10,496] Trial 3 finished with values: [-44.644376708285975, 3.950789059353374] and parameters: {'compression_ratio': 44.356671497470195}. 
[I 2025-03-18 00:01:14,788] Trial 4 finished with values: [-44.12687472271536, 5.374487373192316] and parameters: {'compression_ratio': 60.327896189139175}. 
[I 2025-03-18 00:01:19,162] Trial 5 finished with values: [-44.74687341241343, 3.7154855350387717] and parameters: {'compression_ratio': 41.71075627075087}. 
[I 2025-03-18 00:01:23,513] Trial 6 finished with va

...

[I 2025-03-18 20:26:24,099] Trial 0 finished with values: [-40.41194035338041, 7.69218497086772] and parameters: {'compression_ratio': 84.54724129627861}. 
[I 2025-03-18 20:26:27,433] Trial 1 finished with values: [-40.88787895179496, 5.5753257015109465] and parameters: {'compression_ratio': 61.27320836174953}. 
[I 2025-03-18 20:26:30,774] Trial 2 finished with values: [-41.03655580039551, 5.179035372940942] and parameters: {'compression_ratio': 56.92348001397618}. 
[I 2025-03-18 20:26:34,367] Trial 3 finished with values: [-43.36144255563377, 1.705560607780164] and parameters: {'compression_ratio': 18.74469598530549}. 
[I 2025-03-18 20:26:37,628] Trial 4 finished with values: [-40.39458791250663, 7.88123356898642] and parameters: {'compression_ratio': 86.62934239786706}. 
[I 2025-03-18 20:26:41,177] Trial 5 finished with values: [-42.92275888426908, 2.0225435910342995] and parameters: {'compression_ratio': 22.231962528662066}. 
[I 2025-03-18 20:26:44,468] Trial 6 finished with values:

Compressed S12_2.jpg with compression ratio: 7.69


[I 2025-03-18 20:29:16,930] Trial 0 finished with values: [-40.48562121126103, 5.461625142574448] and parameters: {'compression_ratio': 58.23497636955253}. 
[I 2025-03-18 20:29:20,364] Trial 1 finished with values: [-41.9899638378178, 2.6526954434263352] and parameters: {'compression_ratio': 28.296300781031842}. 
[I 2025-03-18 20:29:23,663] Trial 2 finished with values: [-40.05548905242389, 7.545317048474876] and parameters: {'compression_ratio': 80.48789980111655}. 
[I 2025-03-18 20:29:27,143] Trial 3 finished with values: [-41.7314535479182, 2.9360034474267422] and parameters: {'compression_ratio': 31.3172560267207}. 
[I 2025-03-18 20:29:30,437] Trial 4 finished with values: [-40.08709359564354, 7.2665336035226185] and parameters: {'compression_ratio': 77.52241824150109}. 
[I 2025-03-18 20:29:33,811] Trial 5 finished with values: [-40.56260955061852, 5.216222069101289] and parameters: {'compression_ratio': 55.631181755006274}. 
[I 2025-03-18 20:29:37,202] Trial 6 finished with values

Compressed S13_1.jpg with compression ratio: 5.46


[I 2025-03-18 20:32:10,815] Trial 0 finished with values: [-45.77666390171069, 0.5992297153577063] and parameters: {'compression_ratio': 13.823979657531815}. 
[I 2025-03-18 20:32:14,027] Trial 1 finished with values: [-41.96437741572693, 2.1907766768459265] and parameters: {'compression_ratio': 50.5399668681592}. 
[I 2025-03-18 20:32:17,228] Trial 2 finished with values: [-41.705246207857776, 2.5595574683027156] and parameters: {'compression_ratio': 59.01822339816242}. 
[I 2025-03-18 20:32:20,417] Trial 3 finished with values: [-41.78862659902655, 2.434609618395109] and parameters: {'compression_ratio': 56.17094233604724}. 
[I 2025-03-18 20:32:23,674] Trial 4 finished with values: [-42.733424567782606, 1.4833755530832076] and parameters: {'compression_ratio': 34.22332981354284}. 
[I 2025-03-18 20:32:27,383] Trial 5 finished with values: [-46.27540774659212, 0.5248102121340485] and parameters: {'compression_ratio': 12.106077486627456}. 
[I 2025-03-18 20:32:30,553] Trial 6 finished with 